# Implementatation of ridge regression

In [20]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the training data into feature matrix, class labels, and event ids:

In [21]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

Excluding at the moment all the columns that contains NaN (Data set to zero or -999)

In [22]:
def exclude_NaN(tX):
    sort_no_NaN= [1,2,3,7,8,9,10,11,13,14,15,16,17,18,19,20,21,28,29]
    tX_reduced = tX[:,sort_no_NaN]
    return tX_reduced
tX = exclude_NaN(tX)
tX.shape

(250000, 19)

Normalization of the data, to get 0 mean and unit variance

In [ ]:
def normalize(tX):
    return (tX-np.mean(tX,axis=0))/np.std(tX,axis=0)

tX = normalize(tX)

#We verify whether our data have really mean 0 and variance 1
print(np.mean(tX,axis=0))
print(np.std(tX,axis=0))

### Now let us turn to the proper machine learning side of the problem

Function that builds a polynomial basis from each of the data entry (i.e. goes from $\mathbb{R}^N$ to $\mathbb{R}^{N\cdot D + 1}$ for each data point)

In [29]:
def new_build_poly(x, degree):
    """polynomial basis function."""
    # Creates the matrix with the degrees we want to apply to our data x
    # replecating the vector with [1 2 3 4 ... degree] to a matrix
    #degree_mat = np.tile(list(range(0,degree+1)), (len(x), 1))
    #return np.transpose(np.power(x,np.transpose(degree_mat)))
    
    # Only 1 time 1, because we don't need it to be repeated. That's why for M features and a polynomial of degree D,
    # we will get M*D+1 entries.
    X=np.zeros((x.shape[0],(degree)*x.shape[1]+1))
    for i in range(1,degree+1):
        for j in range(x.shape[1]):
            #print((i-1)*(x.shape[1])+j+1)
            X[:,(i-1)*x.shape[1]+j+1]=x[:,j]**i
    X[:,0]=1
    return X

Implementing 4-fold cross-validation in the following two methods

In [24]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [78]:
from costs import compute_mse
from ridge_regression import ridge_regression

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""
    # ***************************************************
    # get k'th subgroup in test, others in train: 
    # ***************************************************
    x_train = np.array(x[k_indices[k-1]])
    y_train = np.array(y[k_indices[k-1]])
    x_test = np.empty((0,x.shape[1]))
    y_test =  np.empty((0,1))
    for k_iter,validation_points in enumerate(k_indices):
        if(k_iter!=k-1):
            x_test=np.append(x_test,x[validation_points],axis=0)
            y_test=np.append(y_test,y[validation_points])
    #print(x_test.shape)
    #print(y_test.shape)
    # ***************************************************
    # form data with polynomial degree:
    # ***************************************************
    x_train_poly = new_build_poly(x_train,degree)
    x_test_poly = new_build_poly(x_test,degree)
    # ***************************************************
    # ridge regression: 
    # ***************************************************
    w_rr = ridge_regression(y_train,x_train_poly,lambda_)
    # ***************************************************
    # calculate the loss for train and test data:
    # ***************************************************
    #print(x_train_poly.dot(w_rr).shape)
    #print(y_train.shape)
    loss_tr = np.sqrt(2*compute_mse(y_train,x_train_poly,w_rr))
    loss_te = np.sqrt(2*compute_mse(y_test,x_test_poly,w_rr))
    return loss_tr, loss_te

Implementation of the ridge regression

In [75]:
def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    return np.linalg.solve(np.dot(tx.T,tx)+lamb*np.identity(tx.shape[1]),np.dot(tx.T,y))#/(2*len(tx))

In [72]:
def run_ridge_regression(y, tX):
    """ridge regression running script."""
    
    # define parameters for our run   
    seed = 6
    degree = 2
    split_ratio = 0.5
    lambdas = np.logspace(-4, 2, 20)   
    weights_arr = np.zeros((len(lambdas),(degree)*tX.shape[1]+1))
    errors = []
    # ***************************************************
    # train on the whole data and use polynomial basis functions
    # ***************************************************
    tX_train = build_poly(tX, degree)
    
    # ***************************************************
    # ridge regression with different lambdas: 
    # ***************************************************
    for i,lambd in enumerate(lambdas):
        #print(weights_arr[i,:].shape, ' ',X_train.shape)
        weights_arr[i,:] = ridge_regression(y,tX_train,lambd)
        
        #rmse_tr[i] = np.sqrt(2*co.compute_loss(y_train,X_train,weights_arr[i,:]))
        #rmse_te[i] = np.sqrt(2*co.compute_loss(y_test,X_test,weights_arr[i,:]))
                
        #print("lambda ={l}, proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
        #        l=lambd,p=ratio, d=degree, tr=rmse_tr[i], te=rmse_te[i]))
    return weights_arr, errors

weights_arr, errors = run_ridge_regression(y, tX)


In [ ]:
from plots import cross_validation_visualization 

def cross_validation_rr():
    seed = 20
    #degrees = range(2,11)
    degrees = np.array([1,2])
    k_fold = 4
    lambdas = np.logspace(-4, 2, 50)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    # ***************************************************
    # cross validation:
    # ***************************************************    
    rmse_best = np.zeros(len(degrees))
    rmse_best_lambda = np.zeros(len(degrees))
    for j,degree in enumerate(degrees):
        print('\n Testing for a polynomial of degree ', degree)
        rmse_tr = np.zeros(len(lambdas))
        rmse_te = np.zeros(len(lambdas))
        print('lambda=',end="")
        for i,lambda_ in enumerate(lambdas):
            print(round(lambda_,6),end=", ")
            loss_tr_tot=0
            loss_te_tot=0
            for k in range(k_fold+1):
                loss_tr_tmp,loss_te_tmp =cross_validation(y,tX,k_indices,k,lambda_,degree)
                loss_tr_tot += loss_tr_tmp
                loss_te_tot += loss_te_tmp
            rmse_tr[i] = loss_tr_tot/k_fold
            rmse_te[i] = loss_te_tot/k_fold
        rmse_best[j] = min(rmse_te)
        rmse_best_lambda[j] = lambdas[int(np.argmin(rmse_te))]
        cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    print('\n',rmse_best_index)
    print(rmse_best_lambda)
    return rmse_best,rmse_best_lambda
    #plt.figure()
    #plt.plot(degrees,rmse_best)

rmse,lambda_ = cross_validation_rr()


 Testing for a polynomial of degree  1
lambda=0.0001, 0.000133, 0.000176, 0.000233, 0.000309, 0.000409, 0.000543, 0.00072, 0.000954, 0.001265, 0.001677, 0.002223, 0.002947, 0.003907, 0.005179, 0.006866, 0.009103, 0.012068, 0.015999, 0.02121, 0.028118, 0.037276, 0.049417, 0.065513, 0.086851, 0.11514, 0.152642, 0.202359, 0.26827, 0.355648, 0.471487, 0.625055, 0.828643, 1.098541, 1.456348, 1.930698, 2.559548, 3.393222, 4.498433, 5.963623, 7.906043, 10.481131, 13.894955, 18.4207, 24.420531, 32.374575, 42.919343, 56.89866, 75.431201, 100.0, 
 Testing for a polynomial of degree  2
lambda=0.0001, 0.000133, 0.000176, 0.000233, 0.000309, 0.000409, 0.000543, 0.00072, 0.000954, 0.001265, 0.001677, 0.002223, 0.002947, 0.003907, 0.005179, 0.006866, 0.009103, 0.012068, 0.015999, 0.02121, 0.028118, 0.037276, 0.049417, 0.065513, 0.086851, 0.11514, 0.152642, 0.202359, 0.26827, 0.355648, 

NameError: name 'lamdba_' is not defined